# Introduction to Data Cleaning in Jupyter Notebook

Welcome to this Jupyter Notebook where we'll dive into the realm of data cleaning using Python. In this notebook, we'll be working with a CSV file that has undergone some initial cleaning steps in Google Sheets. Our goal here is to further refine and prepare the data for analysis through Python's powerful libraries.

## Data Cleaning Steps Completed in Excel
Downloaded and Prepared Data:

### Downloaded a copy of "FWGN Kids Game Day Volunteers March 16 2024."
    Removed all sheets except for "Game Check Out Sheet."
    Renamed the file to "Game List."

### Reorganized Columns:
    Added a new column named "Copy."
    Moved copy numbers from the "Game Name" column to the "Copy" column.

### Corrected Column Names:
    Used the "KidsGameDayLibrary2024.pdf" to correct column names.
    Switched the column names "Time Out" and "Time In" to correspond to the times written on the form.

### Filled Missing Data:
    Manually filled in missing data based on the scanned copy of "KidsGameDayLibrary2024.pdf."
    If a game was clearly checked in or out but one of the fields was left empty, changed the time to be checked out for one minute.

### Verified Game Names:
    Verified the names of handwritten games using BoardGameGeek.com when possible.
    
### Handled Missing Names:
    Added "Guest" to the name field when a name was missing.

In [1]:
#importing libraries
import pandas as pd
import numpy as np

In [3]:
#reading in data
df = pd.read_csv('clean_data_anonymized.csv') #ananonymized version of the file 

In [ ]:
#exploring data to see what the imported file looks like
df.head()

In [ ]:
df.tail()

In [ ]:
#checking the date types to see what needs to be changed
df.info()

In [9]:
#adding the date of the event to the time columns
#including a space after the date to simplify the concat that will happen in the future
#creating date
date = '2024-03-16 '

In [11]:
#for the 'Time In' column, if there is a 'Time In' value, adding the date before that value
#this will make sure null values stay null while also setting up the column to correct the data type in the future
df['Time In'] = df['Time In'].apply(lambda x: date + x if pd.notnull(x) else x)

In [13]:
#doing the same thing for the 'Time Out' column
df['Time Out'] = df['Time Out'].apply(lambda x: date + x if pd.notnull(x) else x)

In [ ]:
#verifing the new columns
df.head()

In [15]:
#seeing if there are any value changes in data types and null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Temp_ID_Number  123 non-null    int64  
 1   Game Name       123 non-null    object 
 2   Copy            122 non-null    float64
 3   Time Out        80 non-null     object 
 4   Time In         80 non-null     object 
 5   Name            80 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 5.9+ KB


In [17]:
#investigating the 'Copy' column to prepare to change the data type to int
df['Copy'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 123 entries, 0 to 122
Series name: Copy
Non-Null Count  Dtype  
--------------  -----  
122 non-null    float64
dtypes: float64(1)
memory usage: 1.1 KB


In [19]:
#checking the values in the 'Copy' column
df['Copy'].value_counts()

Copy
1.0    117
2.0      4
3.0      1
Name: count, dtype: int64

In [21]:
#finding the record that has a null value for the copy#
missing_mask = df['Copy'].isnull()
df[missing_mask]

,Temp_ID_Number,Game Name,Copy,Time Out,Time In,Name
33,28,Go Fish,NaN,NaN,NaN,NaN


In [23]:
#correcting the 'Copy' number - Since there is only one copy of Go Fish in the lending library, this will be copy #1
df.at[33, 'Copy'] = 1

In [25]:
df.loc[33]

Temp_ID_Number         28
Game Name         Go Fish
Copy                  1.0
Time Out              NaN
Time In               NaN
Name                  NaN
Name: 33, dtype: object

In [27]:
#int would be a better data type for copy# since it will always be in whole numbers
df['Copy'] = df['Copy'].astype(int)

In [29]:
#grabbing column names
df.columns

Index(['Temp_ID_Number', 'Game Name', 'Copy', 'Time Out', 'Time In', 'Name'], dtype='object')

In [31]:
#converting the 'Time Out' column to datetime 
df['Time Out'] = pd.to_datetime(df['Time Out'], format='%Y-%m-%d %H:%M')

In [33]:
#exploring the 'Time In' column since the conversion didn't work
df['Time In'].unique()

array([nan, '2024-03-16 19:00', '2024-03-16 13:25', '2024-03-16 14:46',
       '2024-03-16 16:15', '2024-03-16 12:50', '2024-03-16 16:30',
       '2024-03-16 14:50', '2024-03-16 12:27', '2024-03-16 12:45',
       '2024-03-16 13:05', '2024-03-16 13:55', '2024-03-16 17:15',
       '2024-03-16 13:00', '2024-03-16 14:36', '2024-03-16 17:45',
       '2024-03-16 13:40', '2024-03-16 12:12', '2024-03-16 14:30',
       '2024-03-16 13:12', '2024-03-16 15:08', '2024-03-16 14:00',
       '2024-03-16 14:28', '2024-03-16 12:04', '2024-03-16 14:31',
       '2024-03-16 16:46', '2024-03-16 14:07', '2024-03-16 1600',
       '2024-03-16 1300', '2024-03-16 14:22', '2024-03-16 13:10',
       '2024-03-16 15:47', '2024-03-16 11:27', '2024-03-16 11:35',
       '2024-03-16 17:20', '2024-03-16 10:57', '2024-03-16 14:12',
       '2024-03-16 12:21', '2024-03-16 17:31', '2024-03-16 15:39',
       '2024-03-16 15:18', '2024-03-16 17:43', '2024-03-16 14:57',
       '2024-03-16 10:53', '2024-03-16 14:05', '2024-03-16 

In [35]:
#searching for the first incorrect time
time_search = df['Time In'] == '2024-03-16 1600'
df[time_search]

,Temp_ID_Number,Game Name,Copy,Time Out,Time In,Name
48,39,King of Tokyo,1,2024-03-16 15:55:00,2024-03-16 1600,C Swanson


In [37]:
#correcting the time
df.at[48, 'Time In'] = '2024-03-16 16:00'

In [39]:
#searching for the next incorrect time
time_search = df['Time In'] == '2024-03-16 1300'
df[time_search]

,Temp_ID_Number,Game Name,Copy,Time Out,Time In,Name
54,45,Mancala,1,2024-03-16 12:05:00,2024-03-16 1300,Elisa Randall


In [41]:
#changing it to the correct time
df.at[54, 'Time In'] = '2024-03-16 13:00'

In [43]:
#now updating the 'Time In' column to the correct data type
df['Time In'] = pd.to_datetime(df['Time In'], format='%Y-%m-%d %H:%M')

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Temp_ID_Number  123 non-null    int64         
 1   Game Name       123 non-null    object        
 2   Copy            123 non-null    int32         
 3   Time Out        80 non-null     datetime64[ns]
 4   Time In         80 non-null     datetime64[ns]
 5   Name            80 non-null     object        
dtypes: datetime64[ns](2), int32(1), int64(1), object(2)
memory usage: 5.4+ KB


In [47]:
#checking the values for the Name column
names = sorted(df['Name'].dropna().unique())

In [ ]:
#checking the names to see if any of them need to be corrected
names

In [49]:
len(names)

47

In [51]:
#creating a list of names to be fixed
#for PII issues, changed all the name values
fix_names = ['Anon1', 'Anon2', 'Anon3', 'Anon4', 'Anon5', 'Anon6', 'Anon7',
            'Anon8', 'Anon9']

In [53]:
#all the rows that need to be corrected
f_rows = [row for index, row in df.iterrows() if row['Name'] in fix_names]

In [ ]:
filtered_df = pd.DataFrame(f_rows)
filtered_df

In [57]:
#correcting values
df.at[11, 'Name'] = 'Anon1'
df.at[25, 'Name'] = 'Anon2'
df.at[13, 'Name'] = 'Anon3'
df.at[48, 'Name'] = 'Anon4'
df.at[22, 'Name'] = 'Anon5'
df.at[45, 'Name'] = 'Anon6'
df.at[43, 'Name'] = 'Anon7'
df.at[75, 'Name'] = 'Anon8'
df.at[117, 'Name'] = 'Anon9'

In [59]:
#checking names
names = sorted(df['Name'].dropna().unique())
len(names)

38

In [ ]:
names

In [ ]:
df.head()

In [63]:
#dropping column that was used in excel only
df.drop(columns=['Temp_ID_Number'], inplace=True)

In [ ]:
#downloading a copy of the clean data for tableau
df.to_csv('clean_data.csv', index=True)